In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn.functional as F 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
train = pd.read_csv("C:/Users/moham/OneDrive/Documents/Desktop/JIT/letter_train.csv")
train.replace(r'^\s*$', np.nan, regex=True, inplace=True)
train.dropna(inplace=True)


test = pd.read_csv("C:/Users/moham/OneDrive/Documents/Desktop/JIT/letter_test.csv")
test.replace(r'^\s*$', np.nan, regex=True, inplace=True)
test.dropna(inplace=True)
print(len(test.columns))

64


In [3]:
device = (
    "cpu"
)
print (device)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(len(train.columns)-2,1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,364),
            nn.ReLU(),
            nn.Linear(364,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,29),
            # nn.ReLU(),
            # nn.Linear(32,29)
        )
    def forward(self,x):
        logits=self.layers(x)

        return logits

cpu


In [4]:
model=NeuralNetwork()

In [5]:
import torch.optim as optim
valid_loader= torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
classes=("call","stop","left","right","dislike","like")

In [ ]:
loss_fn=torch.nn.CrossEntropyLoss()
opt=optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
from torch.utils.data import Dataset

class GestureDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # Map 'gesture' and 'Hand_Id' to numerical values
        self.label_mapping = {gesture: idx for idx, gesture in enumerate(self.data['gesture'].unique())}
        self.hand_mapping = {'Left': 0, 'Right': 1}  # Map Hand_Id to numerical values
        self.data['gesture'] = self.data['gesture'].map(self.label_mapping)
        self.data['Hand_Id'] = self.data['Hand_Id'].map(self.hand_mapping)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Include 'Hand_Id' as part of the input features
        hand_id = self.data.iloc[idx, 1]  # Get the 'Hand_Id' value
        features = self.data.iloc[idx, 2:].values.astype(float)  # All columns except 'gesture'
        features = torch.tensor(features, dtype=torch.float32)
        #features = torch.cat((torch.tensor([hand_id], dtype=torch.float32), features))  # Include 'Hand_Id'

        # Labels (gesture)
        label = torch.tensor(self.data.iloc[idx, 0], dtype=torch.long)  # 'gesture' column
                
        return features, label
tr=GestureDataset(train)
train_loder= torch.utils.data.DataLoader(tr, batch_size=40, shuffle=True)
test=GestureDataset(test)
test_loader=torch.utils.data.DataLoader(test)


In [ ]:
def acc(y_pred,y):
    _,pred=torch.max(y_pred,1)
    yes=(pred==y).sum().item()
    return yes/y.size(0)

In [ ]:
from sklearn.preprocessing import StandardScaler
for epoch in range (70):
    ls=0
    model.train()
    for batch,(X,y) in enumerate(train_loder):
        X,y=X.to(device),y.to(device)

        mean=torch.nanmean(X)   
        X=torch.nan_to_num(X,nan=mean)
        #print(X)
        # print(f"Max value in X: {X.max()}")
        # print(f"Min value in X: {X.min()}")
        # print(f"Max value in y: {y.max()}")
        # print(f"Min value in y: {y.min()}")
       # X = (X - X.mean()) / X.std()

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        log=model(X)
        print("NaNs in model output:", torch.isnan(log).sum())
        #print(y)
        loss=loss_fn(log,y)
        print("NaNs in loss:", torch.isnan(loss).sum())
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        print("batch number",batch)
        print(acc(log,y))
        print("acc",loss.item())
        opt.step()
        ls+=loss.item()


In [ ]:
model.eval()

In [ ]:
sum=0
i=0
with torch.no_grad():
    for _,(x,y) in enumerate(test_loader):
        x,y=x.to(device),y.to(device)
        y_pred=model(x)
        accuracy=acc(y_pred,y)
        print("loss",loss_fn(y_pred,y))
        sum+=accuracy
        i+=1
        print(accuracy)
print("tot",(sum/i))

In [ ]:
#saving model
torch.save(model,'model_f.pt')
